In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seq2seq-add/seq2seq_state_add.bin
/kaggle/input/seq2seq-add/vocab_add.bin


In [4]:
import torch
import pickle
from seq2seq_atten_add_model import Seq2Seq


In [6]:
# 加载训练好的模型和词典
state_dict = torch.load('/kaggle/input/seq2seq-add/seq2seq_state_add.bin')
with open('/kaggle/input/seq2seq-add/vocab_add.bin','rb') as f:
    evoc,dvoc = pickle.load(f)

model = Seq2Seq(
    enc_emb_size=len(evoc),
    dec_emb_size=len(dvoc),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.load_state_dict(state_dict)


/tmp/ipykernel_31/3739112445.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/input/seq2seq-add/seq2seq_state_add.bin')
/usr/local/lib/py

<All keys matched successfully>

In [7]:
def infer(enc_input):
    # 创建解码器反向字典
    dvoc_inv = {v:k for k,v in dvoc.items()}
    
    # 用户输入
    # enc_input = "玉 液"
    # enc_input = "What I'm about to say is strictly between you and me"
    # enc_input = "I used to go swimming in the sea when I was a child"
    # enc_input = enc_input.replace(',','').replace('.','').replace('!','').replace('?','')
    
    enc_idx = torch.tensor([[evoc[tk] for tk in enc_input.split(" ")]])
    
    
    # 推理
    # 最大解码长度
    max_dec_len = 50
    
    model.eval()
    with torch.no_grad():
        # 编码器
        # hidden_state = model.encoder(enc_idx)
        hidden_state, enc_outputs = model.encoder(enc_idx)  # attention
        
        # 解码器输入 shape [1,1]
        dec_input = torch.tensor([[dvoc['BOS']]])
        
        # 循环decoder
        dec_tokens = []
    while True:
        if len(dec_tokens) >= enc_idx.shape[1]:
            break
        # 解码器 
        # logits: [1,1,dec_voc_size]
        # logits,hidden_state = model.decoder(dec_input, hidden_state)
        logits,hidden_state = model.decoder(dec_input, hidden_state, enc_outputs) ##ATTENTION 
        
        # 下个token index
        next_token = torch.argmax(logits, dim=-1)
        if dvoc_inv[next_token.squeeze().item()] == 'EOS':
            break
        # 收集每次token_index 【解码集合】
        dec_tokens.append(next_token.squeeze().item())
        # decoder的下一个输入 = token_index
        dec_input = next_token
        hidden_state = hidden_state.view(1, -1)
    
    # 输出解码结果
    return ' '.join([dvoc_inv[tk] for tk in dec_tokens])

In [8]:
print("上 联   ：",'古 典 精 华 凝 诗 句')
print("下 联   ：",'现 代 风 貌 入 画 屏')
print("推理下联 ：",infer('古 典 精 华 凝 诗 句'))

上 联   ： 古 典 精 华 凝 诗 句
下 联   ： 现 代 风 貌 入 画 屏
推理下联 ： 春 风 雨 露 润 诗 情


In [9]:
print("上 联   ：",'风 声 雨 声 读 书 声 声 声 入 耳')
print("下 联   ：",'家 事 国 事 天 下 事 事 事 关 心')
print("推理下联 ：",infer('风 声 雨 声 读 书 声 声 声 入 耳'))



上 联   ： 风 声 雨 声 读 书 声 声 声 入 耳
下 联   ： 家 事 国 事 天 下 事 事 事 关 心
推理下联 ： 水 秀 山 水 画 图 画 卷 画 图 画


In [11]:
print("上 联   ：",'南 通 州 ， 北 通 州 ， 南 北 通 州 通 南 北')
print("下 联   ：",'春 读 书 ， 秋 读 书 ， 春 秋 读 书 读 春 秋')
print("推理下联 ：",infer('南 通 州 ， 北 通 州 ， 南 北 通 州 通 南 北'))


上 联   ： 南 通 州 ， 北 通 州 ， 南 北 通 州 通 南 北
下 联   ： 春 读 书 ， 秋 读 书 ， 春 秋 读 书 读 春 秋
推理下联 ： 北 京 城 ， 南 北 京 ， 东 西 南 北 东 西 南
